In [1]:
import spotipy
import spotipy as spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

import pprint
pp = pprint.PrettyPrinter(indent=4)


client_id = '12e4e34e23e841b3aa386f3e1e9afa74'
client_secret = '1e5335d90d44414495573690e004d3ae'
# client_id = 'dab809ffa57e4712981d068a2dbcc8f6'
# client_secret = 'd0e73dc83d4d45e7aaea970f95de6c23'

client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [2]:
df = pd.read_csv('./dataset/unique_min_rank.csv', header=0, encoding = "ISO-8859-1")
df = df.drop(['Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'], axis=1)
df.head()

/tmp/ipykernel_39452/2507618338.py:1: DtypeWarning: Columns (11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset/unique_min_rank.csv', header=0, encoding = "ISO-8859-1")


,title,rank,date,artist,url,region,chart,trend,streams,min_rank
0,23 (With Ape Drums),157,10/22/2021,Randy,https://open.spotify.com/track/4vv8wcnT04GHHbU...,Argentina,top200,NEW_ENTRY,37919,76
1,24K,116,9/9/2019,"Frijo, Santoz, Duki",https://open.spotify.com/track/1sbxPVKNi6OLUdT...,Argentina,top200,NEW_ENTRY,31528,108
2,4:00 AM,193,2/23/2018,Trueno,https://open.spotify.com/track/2rN6ZDtVTPflMgf...,Argentina,top200,NEW_ENTRY,16786,193
3,Agua (with J Balvin),145,3/5/2021,Tainy,https://open.spotify.com/track/7CtbxAGq2P0jtYh...,Argentina,top200,MOVE_UP,33015,124
4,Ahora mirame,199,7/1/2017,Ulises Bueno,https://open.spotify.com/track/780CrNUSw5RrRWH...,Argentina,top200,NEW_ENTRY,13323,153


In [3]:
print(df.shape)
df = df.dropna()
df.shape

(299532, 10)


(296597, 10)

In [4]:
from tqdm import tqdm

tqdm.pandas()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [9]:
def get_all_features(df_row):
    global row_cnt, failed_cnt
    row_cnt += 1
    if row_cnt % 1000 == 0:
        print(f"Row Count: {row_cnt}")
    try:
        features = sp.audio_features(df_row['url'])[0]
        df_row['danceability'] = features['danceability']
        df_row['energy'] = features['energy']
        df_row['key'] = features['key']
        df_row['loudness'] = features['loudness']
        df_row['mode'] = features['mode']
        df_row['speechiness'] = features['speechiness']
        df_row['acousticness'] = features['acousticness']
        df_row['instrumentalness'] = features['instrumentalness']
        df_row['liveness'] = features['liveness']
        df_row['valence'] = features['valence']
        df_row['tempo'] = features['tempo']
        df_row['duration_ms'] = features['duration_ms']

        try:
            track_info = sp.track(df_row['url'])
            df_row['artist_url'] = track_info['artists'][0]['external_urls']['spotify']
            df_row['track_popularity'] = track_info['popularity']

            try:
                artist_info = sp.artist(df_row['artist_url'])
                df_row['artist_genres'] = artist_info['genres']
                df_row['artist_popularity'] = artist_info['popularity']

            except Exception as e:
                print(e)
                failed_cnt += 1
        except Exception as e:
            print(e)
            failed_cnt += 1
    except Exception as e:
        print(e)
        failed_cnt += 1

    if failed_cnt > 0 and failed_cnt % 1000 == 0:
        print(f"Failed Count: {failed_cnt}")

    return df_row

In [10]:
def get_all_chunk_features(df_rows):
    global row_cnt, failed_cnt
    row_cnt += 1
    if row_cnt % 1000 == 0:
        print(f"Row Count: {row_cnt}")
    try:
        # if row_cnt == 1:
        #     print(df_rows.head(10))
        chunk_features = sp.audio_features(list(df_rows['url']))
        for i in range(len(chunk_features)):
            df_rows.iloc[i]['danceability'] = chunk_features[i]['danceability']
            df_rows.iloc[i]['energy'] = chunk_features[i]['energy']
            df_rows.iloc[i]['key'] = chunk_features[i]['key']
            df_rows.iloc[i]['loudness'] = chunk_features[i]['loudness']
            df_rows.iloc[i]['mode'] = chunk_features[i]['mode']
            df_rows.iloc[i]['speechiness'] = chunk_features[i]['speechiness']
            df_rows.iloc[i]['acousticness'] = chunk_features[i]['acousticness']
            df_rows.iloc[i]['instrumentalness'] = chunk_features[i]['instrumentalness']
            df_rows.iloc[i]['liveness'] = chunk_features[i]['liveness']
            df_rows.iloc[i]['valence'] = chunk_features[i]['valence']
            df_rows.iloc[i]['tempo'] = chunk_features[i]['tempo']
            df_rows.iloc[i]['duration_ms'] = chunk_features[i]['duration_ms']

        try:
            chunk_track_info = sp.tracks(list(df_rows['url']))
            for i in range(len(chunk_track_info)):
                df_rows.iloc[i]['artist_url'] = chunk_track_info[i]['artists'][0]['external_urls']['spotify']
                df_rows.iloc[i]['track_popularity'] = chunk_track_info[i]['popularity']

            try:
                chunk_artist_info = sp.artist(list(df_rows['artist_url']))
                for i in range(len(chunk_track_info)):
                    df_rows.iloc[i]['artist_genres'] = chunk_artist_info[i]['genres']
                    df_rows.iloc[i]['artist_popularity'] = chunk_artist_info[i]['popularity']

            except Exception as e:
                print(e)
                failed_cnt += 1
        except Exception as e:
            print(e)
            failed_cnt += 1
    except Exception as e:
        print(e)
        failed_cnt += 1

    if failed_cnt > 0 and failed_cnt % 1000 == 0:
        print(f"Failed Count: {failed_cnt}")

    return df_rows

In [1]:
import signal

def handle_timeout(signum, frame):
    # This function will be called when the timeout is reached.
    raise Exception("Timeout reached")

def get_all_features_with_timeout(row):
    # Set a timeout of 10 seconds.
    signal.signal(signal.SIGALRM, handle_timeout)
    signal.alarm(10)

    try:
        result = get_all_features(row)
        signal.alarm(0)  # Cancel the timeout.
        return result
    except Exception as e:
        print(e)
        signal.alarm(0)  # Cancel the timeout.
        return row

def get_all_chunk_features_with_timeout(rows):
    # Set a timeout of 10 seconds.
    signal.signal(signal.SIGALRM, handle_timeout)
    signal.alarm(120)

    try:
        result = get_all_chunk_features(rows)
        signal.alarm(0)  # Cancel the timeout.
        return result
    except Exception as e:
        print(e)
        signal.alarm(0)  # Cancel the timeout.
        return rows

In [12]:
import numpy as np
splitted_df = np.array_split(df, 1000)
print(len(splitted_df))
print(splitted_df[0].shape)

1000
(297, 10)


In [13]:
split_idx = [i for i in range(1000)]
for i in split_idx:
    row_cnt = 0
    failed_cnt = 0
    chunks = np.array_split(splitted_df[i], 6)
    for j in range(6):
        chunks[j] = get_all_chunk_features_with_timeout(chunks[j])
    # splitted_df[i] = splitted_df[i].progress_apply(get_all_features_with_timeout, axis=1)
    result_df = pd.concat(chunks, axis=0)
    print(f"Row Count: {row_cnt}, Failed Count: {failed_cnt}")
    result_df.to_csv(f"./outputs/final_outputs_{i}.csv", encoding='utf-8', index=False)

Timeout reached


KeyboardInterrupt: 

: 

In [ ]:
df.to_csv('./final_outputs.csv', encoding='utf-8', index=False)

In [11]:
sp.audio_features(['https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF', 'https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF'])[0]

{'danceability': 0.828,
 'energy': 0.724,
 'key': 6,
 'loudness': -3.555,
 'mode': 0,
 'speechiness': 0.0715,
 'acousticness': 0.0424,
 'instrumentalness': 1.68e-06,
 'liveness': 0.468,
 'valence': 0.896,
 'tempo': 96.058,
 'type': 'audio_features',
 'id': '4vv8wcnT04GHHbUCFBAiHF',
 'uri': 'spotify:track:4vv8wcnT04GHHbUCFBAiHF',
 'track_href': 'https://api.spotify.com/v1/tracks/4vv8wcnT04GHHbUCFBAiHF',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4vv8wcnT04GHHbUCFBAiHF',
 'duration_ms': 167504,
 'time_signature': 4}

In [5]:
temp_df = df.iloc[:100, :]
temp_df.shape

(100, 10)

In [19]:
def get_audio_features(df_row):
    global row_cnt, success_cnt
    row_cnt += 1
    if row_cnt % 1000 == 0:
        print(f"Row Count: {row_cnt}")
    try:
        features = sp.audio_features(df_row['url'])[0]
        df_row['danceability'] = features['danceability']
        df_row['energy'] = features['energy']
        df_row['key'] = features['key']
        df_row['loudness'] = features['loudness']
        df_row['mode'] = features['mode']
        df_row['speechiness'] = features['speechiness']
        df_row['acousticness'] = features['acousticness']
        df_row['instrumentalness'] = features['instrumentalness']
        df_row['liveness'] = features['liveness']
        df_row['valence'] = features['valence']
        df_row['tempo'] = features['tempo']
        df_row['duration_ms'] = features['duration_ms']
        success_cnt += 1
        if success_cnt % 1000 == 0:
            print(f"Success Count: {success_cnt}")
    except:
        pass
    
    return df_row

In [24]:
row_cnt = 0
success_cnt = 0
temp_df = temp_df.progress_apply(get_audio_features, axis=1)
temp_df.head(10)

100%|██████████| 100/100 [00:02<00:00, 38.88it/s]


,title,rank,date,artist,url,region,chart,trend,streams,min_rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,23 (With Ape Drums),157,10/22/2021,Randy,https://open.spotify.com/track/4vv8wcnT04GHHbU...,Argentina,top200,NEW_ENTRY,37919,76,0.828,0.724,6,-3.555,0,0.0715,0.0424,0.000002,0.4680,0.896,96.058,167504
1,24K,116,9/9/2019,"Frijo, Santoz, Duki",https://open.spotify.com/track/1sbxPVKNi6OLUdT...,Argentina,top200,NEW_ENTRY,31528,108,0.810,0.833,1,-6.832,1,0.1250,0.2530,0.005760,0.2160,0.907,138.955,123808
2,4:00 AM,193,2/23/2018,Trueno,https://open.spotify.com/track/2rN6ZDtVTPflMgf...,Argentina,top200,NEW_ENTRY,16786,193,0.804,0.590,11,-8.524,0,0.2640,0.1700,0.000000,0.1060,0.453,123.947,247339
3,Agua (with J Balvin),145,3/5/2021,Tainy,https://open.spotify.com/track/7CtbxAGq2P0jtYh...,Argentina,top200,MOVE_UP,33015,124,0.778,0.651,9,-5.421,0,0.2660,0.1670,0.000000,0.1770,0.569,100.094,159666
4,Ahora mirame,199,7/1/2017,Ulises Bueno,https://open.spotify.com/track/780CrNUSw5RrRWH...,Argentina,top200,NEW_ENTRY,13323,153,0.735,0.917,11,-3.962,0,0.0389,0.0673,0.000000,0.0453,0.811,149.961,274213
5,All The Stars (with SZA),190,2/9/2018,Kendrick Lamar,https://open.spotify.com/track/2Imq77E6CEJR5TP...,Argentina,top200,NEW_ENTRY,16524,124,0.532,0.642,5,-4.891,0,0.1110,0.0548,0.000246,0.0991,0.536,193.016,232187
6,All of Me,193,2/6/2017,John Legend,https://open.spotify.com/track/2q6tgiA9tpb6DMg...,Argentina,top200,NEW_ENTRY,11296,89,0.422,0.264,8,-7.064,1,0.0322,0.9220,0.000000,0.1320,0.331,119.930,269560
7,Alone,190,1/30/2017,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Argentina,top200,NEW_ENTRY,12091,183,0.676,0.929,10,-3.194,1,0.0458,0.1860,0.000405,0.1210,0.157,97.019,161200
8,Another Day in Paradise - 2016 Remaster,188,3/21/2018,Phil Collins,https://open.spotify.com/track/1NCuYqMc8hKMb4c...,Argentina,top200,NEW_ENTRY,15307,188,0.780,0.565,3,-7.320,0,0.0306,0.7360,0.002690,0.0630,0.365,101.967,322133
9,"Asan: Bzrp Music Sessions, Vol. 35",69,11/12/2020,"Bizarrap, Asan",https://open.spotify.com/track/5ZoYsrmeVPyJI4r...,Argentina,top200,NEW_ENTRY,54640,69,0.680,0.576,8,-8.402,1,0.0866,0.2050,0.000000,0.1090,0.507,164.978,154182


In [5]:
sp.track('https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF')

KeyboardInterrupt: 

In [12]:
print(sp.track('https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF')['artists'][0]['external_urls']['spotify'])
print(sp.track('https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF')['popularity'])

https://open.spotify.com/artist/7qYeIN2r4H1kBvr0Gm9Iav
67


In [25]:
def get_track_info(df_row):
    global row_cnt, success_cnt
    row_cnt += 1
    if row_cnt % 1000 == 0:
        print(f"Row Count: {row_cnt}")
    try:
        track_info = sp.track(df_row['url'])
        df_row['artist_url'] = track_info['artists'][0]['external_urls']['spotify']
        df_row['track_popularity'] = track_info['popularity']
        success_cnt += 1
        if success_cnt % 1000 == 0:
            print(f"Success Count: {success_cnt}")
    except:
        pass
    
    return df_row

In [27]:
row_cnt = 0
success_cnt = 0
temp_df = temp_df.progress_apply(get_track_info, axis=1)
temp_df.head(10)

100%|██████████| 100/100 [00:05<00:00, 17.67it/s]


,title,rank,date,artist,url,region,chart,trend,streams,min_rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_url,track_popularity
0,23 (With Ape Drums),157,10/22/2021,Randy,https://open.spotify.com/track/4vv8wcnT04GHHbU...,Argentina,top200,NEW_ENTRY,37919,76,0.828,0.724,6,-3.555,0,0.0715,0.0424,0.000002,0.4680,0.896,96.058,167504,https://open.spotify.com/artist/7qYeIN2r4H1kBv...,67
1,24K,116,9/9/2019,"Frijo, Santoz, Duki",https://open.spotify.com/track/1sbxPVKNi6OLUdT...,Argentina,top200,NEW_ENTRY,31528,108,0.810,0.833,1,-6.832,1,0.1250,0.2530,0.005760,0.2160,0.907,138.955,123808,https://open.spotify.com/artist/4D2d63igYEdzhz...,60
2,4:00 AM,193,2/23/2018,Trueno,https://open.spotify.com/track/2rN6ZDtVTPflMgf...,Argentina,top200,NEW_ENTRY,16786,193,0.804,0.590,11,-8.524,0,0.2640,0.1700,0.000000,0.1060,0.453,123.947,247339,https://open.spotify.com/artist/2x7PC78TmgqpEI...,0
3,Agua (with J Balvin),145,3/5/2021,Tainy,https://open.spotify.com/track/7CtbxAGq2P0jtYh...,Argentina,top200,MOVE_UP,33015,124,0.778,0.651,9,-5.421,0,0.2660,0.1670,0.000000,0.1770,0.569,100.094,159666,https://open.spotify.com/artist/0GM7qgcRCORpGn...,67
4,Ahora mirame,199,7/1/2017,Ulises Bueno,https://open.spotify.com/track/780CrNUSw5RrRWH...,Argentina,top200,NEW_ENTRY,13323,153,0.735,0.917,11,-3.962,0,0.0389,0.0673,0.000000,0.0453,0.811,149.961,274213,https://open.spotify.com/artist/2UqRkW2wfEkZmy...,0
5,All The Stars (with SZA),190,2/9/2018,Kendrick Lamar,https://open.spotify.com/track/2Imq77E6CEJR5TP...,Argentina,top200,NEW_ENTRY,16524,124,0.532,0.642,5,-4.891,0,0.1110,0.0548,0.000246,0.0991,0.536,193.016,232187,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,0
6,All of Me,193,2/6/2017,John Legend,https://open.spotify.com/track/2q6tgiA9tpb6DMg...,Argentina,top200,NEW_ENTRY,11296,89,0.422,0.264,8,-7.064,1,0.0322,0.9220,0.000000,0.1320,0.331,119.930,269560,https://open.spotify.com/artist/5y2Xq6xcjJb2jV...,0
7,Alone,190,1/30/2017,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Argentina,top200,NEW_ENTRY,12091,183,0.676,0.929,10,-3.194,1,0.0458,0.1860,0.000405,0.1210,0.157,97.019,161200,https://open.spotify.com/artist/7vk5e3vY1uw9pl...,62
8,Another Day in Paradise - 2016 Remaster,188,3/21/2018,Phil Collins,https://open.spotify.com/track/1NCuYqMc8hKMb4c...,Argentina,top200,NEW_ENTRY,15307,188,0.780,0.565,3,-7.320,0,0.0306,0.7360,0.002690,0.0630,0.365,101.967,322133,https://open.spotify.com/artist/4lxfqrEsLX6N1N...,74
9,"Asan: Bzrp Music Sessions, Vol. 35",69,11/12/2020,"Bizarrap, Asan",https://open.spotify.com/track/5ZoYsrmeVPyJI4r...,Argentina,top200,NEW_ENTRY,54640,69,0.680,0.576,8,-8.402,1,0.0866,0.2050,0.000000,0.1090,0.507,164.978,154182,https://open.spotify.com/artist/716NhGYqD1jl2w...,9


In [11]:
sp.artist('https://open.spotify.com/artist/7qYeIN2r4H1kBvr0Gm9Iav')

KeyboardInterrupt: 

In [32]:
def get_artist_info(df_row):
    global row_cnt, success_cnt
    row_cnt += 1
    if row_cnt % 1000 == 0:
        print(f"Row Count: {row_cnt}")
    try:
        artist_info = sp.artist(df_row['artist_url'])
        df_row['artist_genres'] = artist_info['genres']
        df_row['artist_popularity'] = artist_info['popularity']
        success_cnt += 1
        if success_cnt % 1000 == 0:
            print(f"Success Count: {success_cnt}")
    except:
        pass
    
    return df_row

In [33]:
row_cnt = 0
success_cnt = 0
temp_df = temp_df.progress_apply(get_artist_info, axis=1)
temp_df.head(10)

100%|██████████| 100/100 [00:03<00:00, 32.58it/s]


,title,rank,date,artist,url,region,chart,trend,streams,min_rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_url,track_popularity,artist_genres,artist_popularity
0,23 (With Ape Drums),157,10/22/2021,Randy,https://open.spotify.com/track/4vv8wcnT04GHHbU...,Argentina,top200,NEW_ENTRY,37919,76,0.828,0.724,6,-3.555,0,0.0715,0.0424,0.000002,0.4680,0.896,96.058,167504,https://open.spotify.com/artist/7qYeIN2r4H1kBv...,67,"[latin hip hop, perreo, reggaeton, reggaeton f...",67
1,24K,116,9/9/2019,"Frijo, Santoz, Duki",https://open.spotify.com/track/1sbxPVKNi6OLUdT...,Argentina,top200,NEW_ENTRY,31528,108,0.810,0.833,1,-6.832,1,0.1250,0.2530,0.005760,0.2160,0.907,138.955,123808,https://open.spotify.com/artist/4D2d63igYEdzhz...,60,"[argentine hip hop, trap argentino]",57
2,4:00 AM,193,2/23/2018,Trueno,https://open.spotify.com/track/2rN6ZDtVTPflMgf...,Argentina,top200,NEW_ENTRY,16786,193,0.804,0.590,11,-8.524,0,0.2640,0.1700,0.000000,0.1060,0.453,123.947,247339,https://open.spotify.com/artist/2x7PC78TmgqpEI...,0,"[argentine hip hop, bases de freestyle, trap a...",74
3,Agua (with J Balvin),145,3/5/2021,Tainy,https://open.spotify.com/track/7CtbxAGq2P0jtYh...,Argentina,top200,MOVE_UP,33015,124,0.778,0.651,9,-5.421,0,0.2660,0.1670,0.000000,0.1770,0.569,100.094,159666,https://open.spotify.com/artist/0GM7qgcRCORpGn...,67,"[latin hip hop, pop reggaeton, reggaeton, regg...",77
4,Ahora mirame,199,7/1/2017,Ulises Bueno,https://open.spotify.com/track/780CrNUSw5RrRWH...,Argentina,top200,NEW_ENTRY,13323,153,0.735,0.917,11,-3.962,0,0.0389,0.0673,0.000000,0.0453,0.811,149.961,274213,https://open.spotify.com/artist/2UqRkW2wfEkZmy...,0,[cuarteto],61
5,All The Stars (with SZA),190,2/9/2018,Kendrick Lamar,https://open.spotify.com/track/2Imq77E6CEJR5TP...,Argentina,top200,NEW_ENTRY,16524,124,0.532,0.642,5,-4.891,0,0.1110,0.0548,0.000246,0.0991,0.536,193.016,232187,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,0,"[conscious hip hop, hip hop, rap, west coast rap]",90
6,All of Me,193,2/6/2017,John Legend,https://open.spotify.com/track/2q6tgiA9tpb6DMg...,Argentina,top200,NEW_ENTRY,11296,89,0.422,0.264,8,-7.064,1,0.0322,0.9220,0.000000,0.1320,0.331,119.930,269560,https://open.spotify.com/artist/5y2Xq6xcjJb2jV...,0,"[neo mellow, neo soul, pop, pop soul, r&b, urb...",78
7,Alone,190,1/30/2017,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Argentina,top200,NEW_ENTRY,12091,183,0.676,0.929,10,-3.194,1,0.0458,0.1860,0.000405,0.1210,0.157,97.019,161200,https://open.spotify.com/artist/7vk5e3vY1uw9pl...,62,[electro house],81
8,Another Day in Paradise - 2016 Remaster,188,3/21/2018,Phil Collins,https://open.spotify.com/track/1NCuYqMc8hKMb4c...,Argentina,top200,NEW_ENTRY,15307,188,0.780,0.565,3,-7.320,0,0.0306,0.7360,0.002690,0.0630,0.365,101.967,322133,https://open.spotify.com/artist/4lxfqrEsLX6N1N...,74,"[mellow gold, rock drums, soft rock]",77
9,"Asan: Bzrp Music Sessions, Vol. 35",69,11/12/2020,"Bizarrap, Asan",https://open.spotify.com/track/5ZoYsrmeVPyJI4r...,Argentina,top200,NEW_ENTRY,54640,69,0.680,0.576,8,-8.402,1,0.0866,0.2050,0.000000,0.1090,0.507,164.978,154182,https://open.spotify.com/artist/716NhGYqD1jl2w...,9,"[argentine hip hop, pop venezolano, trap argen...",88
